# Librerías

In [77]:
import pandas as pd
import numpy as np
import ast
import json
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import matplotlib.pyplot as plt
import pyarrow
sns.set()

# _________________________________________

# Proceso de ETL

### Proceso de limpieza del dataframe _'users_reviews'_

In [3]:
# Se carga el CSV generado previamente

lista_rows = []

with open('/Users/mlucchesi/Henry/PI/data/csv/df_user_reviews_fixed.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    headers = next(csv_reader)
    for row in csv_reader:
        lista_rows.append(row)

# Se transponen las filas, guardadas en listas, para que queden como columnas
df_user_reviews_fixed = pd.DataFrame(lista_rows, columns=headers)

In [22]:
# Se muestra

df_user_reviews_fixed.head()

,,user_id,user_url,posted,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [5]:
# Eliminar registros donde todos sus valores sean nulos

df_user_reviews_fixed = df_user_reviews_fixed.dropna(how='all')

In [6]:
# Reemplazar los valores vacios por 'NA'

df_user_reviews_fixed.replace('', pd.NA, inplace=True)

In [9]:
''' Elimina los registros con valores nulos en 'item_id' ya que no tendría sentido tener 
el registro de una reseña si no se conoce el 'id' del juego en cuestión '''

df_user_reviews_fixed = df_user_reviews_fixed.dropna(subset=['item_id'])

In [14]:
# Se analizan cuantos nulos hay en cada columna

nulos_users_reviews = (df_user_reviews_fixed.isnull().mean() * 100).round(2)
nulos_users_reviews

                0.00
user_id        53.84
user_url       53.84
funny          85.96
posted          0.00
last_edited    90.10
item_id         0.00
helpful         0.00
recommend       0.00
review          0.05
dtype: float64

In [15]:
''' Ya que en su mayoría, las columnas 'funny' y 'last_edited' son registros nulos,
se eliminan para mejorar la optimización del dataframe '''

df_user_reviews_fixed.drop(['funny', 'last_edited'], axis=1, inplace=True)

In [18]:
df_user_reviews_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55832 entries, 0 to 154739
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0              55832 non-null  object
 1   user_id    25771 non-null  object
 2   user_url   25771 non-null  object
 3   posted     55832 non-null  object
 4   item_id    55832 non-null  object
 5   helpful    55832 non-null  object
 6   recommend  55832 non-null  object
 7   review     55803 non-null  object
dtypes: object(8)
memory usage: 3.8+ MB


In [20]:
# Debido a su relevancia, se cambian los tipos de datos en las columnas 'item_id' y 'recommend'

df_user_reviews_fixed['recommend'] = df_user_reviews_fixed['recommend'].astype(bool)
df_user_reviews_fixed['item_id'] = df_user_reviews_fixed['item_id'].astype(int)

In [21]:
df_user_reviews_fixed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55832 entries, 0 to 154739
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0              55832 non-null  object
 1   user_id    25771 non-null  object
 2   user_url   25771 non-null  object
 3   posted     55832 non-null  object
 4   item_id    55832 non-null  int64 
 5   helpful    55832 non-null  object
 6   recommend  55832 non-null  bool  
 7   review     55803 non-null  object
dtypes: bool(1), int64(1), object(6)
memory usage: 3.5+ MB


In [ ]:
# Elimino la primera columna que había quedado "repitiendo" el índice

df_user_reviews_fixed = df_user_reviews_fixed.drop(columns=[''], axis=1)
df_user_reviews_fixed

In [32]:
# Se exportan los resultados nuevamente a un CSV 

df_user_reviews_fixed.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/users_reviews_procesado.csv')

### Proceso de limpieza del dataframe _'steam_games'_

In [59]:
# Se carga el CSV generado previamente

lista_rows = []

with open('/Users/mlucchesi/Henry/PI/data/csv/df_steam_games.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    headers = next(csv_reader)
    for row in csv_reader:
        lista_rows.append(row)

# Se transponen las filas, guardadas en listas, para que queden como columnas
df_steam_games = pd.DataFrame(lista_rows, columns=headers)

In [60]:
# Muestro el dataframe (los últimos registros ya que los primeros son todos nulos)

df_steam_games.tail()

,,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"['Single-player', 'Steam Achievements']",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,733530.0,Sacada
120442,120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,658870.0,"xropi,stev3ns"
120444,120444,,,Maze Run VR,,http://store.steampowered.com/app/681550/Maze_...,,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,"['Single-player', 'Stats', 'Steam Leaderboards...",4.99,1.0,681550.0,


In [61]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0                 120445 non-null  object
 1   publisher     120445 non-null  object
 2   genres        120445 non-null  object
 3   app_name      120445 non-null  object
 4   title         120445 non-null  object
 5   url           120445 non-null  object
 6   release_date  120445 non-null  object
 7   tags          120445 non-null  object
 8   reviews_url   120445 non-null  object
 9   specs         120445 non-null  object
 10  price         120445 non-null  object
 11  early_access  120445 non-null  object
 12  id            120445 non-null  object
 13  developer     120445 non-null  object
dtypes: object(14)
memory usage: 12.9+ MB


In [62]:
# Elimino la primera columna que había quedado "repitiendo" el índice

df_steam_games = df_steam_games.drop(columns=[''], axis=1)

In [63]:
# Eliminar registros donde todos sus valores sean nulos

df_steam_games = df_steam_games.dropna(how='all')

In [64]:
# Reemplazar los valores vacios por 'NA'

df_steam_games.replace('', pd.NA, inplace=True)

In [65]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [66]:
# Hago un análisis de todos los valores que componen la columna 'price'

prices_sin_na = df_steam_games['price'].dropna()

prices_unique = prices_sin_na.unique()

# Los ordeno para diferenciar los números de los strings
prices_sorted = sorted(prices_unique)

# Mostrar los valores distintos de 'price' ordenados
num_prices = len(prices_sorted)
print("Número total de valores diferentes en 'prices':", num_prices)
print()

for price in prices_sorted:
    print(price)


Número total de valores diferentes en 'prices': 162

0.49
0.5
0.89
0.9500000000000001
0.98
0.99
1.0
1.25
1.29
1.3900000000000001
1.49
1.5
1.5899999999999999
1.87
1.9500000000000002
1.99
10.0
10.49
10.93
10.96
10.99
109.99
11.15
11.99
119.99
12.0
12.89
12.99
124.99
129.99
13.37
13.98
13.99
131.4
139.92
14.95
14.99
149.99
15.0
15.99
16.06
16.99
160.91
17.99
172.24
179.0
18.9
18.99
189.0
189.96
19.29
19.95
19.98
19.99
199.0
199.99
2.0
2.3
2.49
2.66
2.89
2.9699999999999998
2.99
20.0
20.99
202.76
21.99
22.99
23.96
23.99
234.99
24.99
249.99
26.99
27.49
27.99
289.99
29.96
29.99
299.99
3.0
3.33
3.39
3.49
3.99
30.0
31.99
32.99
320.0
34.99
36.99
38.85
39.99
399.0
399.99
4.0
4.29
4.49
4.68
4.99
40.0
41.99
42.99
44.98
44.99
49.0
49.99
499.99
5.0
5.49
5.65
5.99
54.99
59.95
59.99
599.0
6.0
6.48
6.49
6.66
6.99
61.99
64.99
69.99
7.0
7.49
7.99
71.7
74.76
74.99
771.71
79.99
8.98
8.99
87.94
89.99
9.0
9.69
9.95
9.98
9.99
99.0
99.99
995.0
Free
Free Demo
Free HITMAN™ Holiday Pack
Free Mod
Free Movie
Free To

In [67]:
# Reemplazo todos los registros con strings referidos a que un juego es gratis, por el valor '0' (todavía en string)

df_steam_games.replace(['Free', 'Free Demo', 'Free HITMAN™ Holiday Pack', 'Free Mod', 'Free Movie', 'Free To Play', 'Free to Play', 'Free to Try', 'Free to Use', 'Play for Free!'], '0', inplace=True)

In [ ]:
df_steam_games.tail()


In [69]:
# Elimina los valores diferentes a números y los valores nulos de la columna 'price'

df_steam_games = df_steam_games[df_steam_games['price'].notna() & df_steam_games['price'].str.match(r'^-?\d+\.?\d*$')]

In [70]:
# Transformo el tipo de datos de la columna 'price' a float

df_steam_games['price']= df_steam_games['price'].astype(float)

In [71]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30748 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22979 non-null  object 
 1   genres        27614 non-null  object 
 2   app_name      30747 non-null  object 
 3   title         28815 non-null  object 
 4   url           30748 non-null  object 
 5   release_date  28811 non-null  object 
 6   tags          30617 non-null  object 
 7   reviews_url   30746 non-null  object 
 8   specs         30094 non-null  object 
 9   price         30748 non-null  float64
 10  early_access  30748 non-null  object 
 11  id            30746 non-null  object 
 12  developer     27593 non-null  object 
dtypes: float64(1), object(12)
memory usage: 3.3+ MB


In [73]:
# Analizo cuantos valores nulos quedaron por columna luego del proceso de transoformación

nulos_steam_games = (df_steam_games.isnull().mean() * 100).round(2)
nulos_steam_games

publisher       25.27
genres          10.19
app_name         0.00
title            6.29
url              0.00
release_date     6.30
tags             0.43
reviews_url      0.01
specs            2.13
price            0.00
early_access     0.00
id               0.01
developer       10.26
dtype: float64

In [74]:
# Se exportan los resultados nuevamente a un CSV 

df_steam_games.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/steam_games_procesado.csv')

In [78]:
# Se exporta a parquet, por motivos de optimización en la API

df_steam_games.to_parquet('/Users/mlucchesi/Henry/PI/data/parquet/steam_games.parquet', engine='pyarrow')

### Proceso de limpieza del dataframe _'users_items'_

In [79]:
# Se carga el CSV generado previamente

lista_rows = []

with open('/Users/mlucchesi/Henry/PI/data/csv/df_users_items_fixed.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    headers = next(csv_reader)
    for row in csv_reader:
        lista_rows.append(row)

# Se transponen las filas, guardadas en listas, para que queden como columnas
df_users_items = pd.DataFrame(lista_rows, columns=headers)

In [86]:
# Muestro los primeros registros

df_users_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0,0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,30,Day of Defeat,7,0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,40,Deathmatch Classic,0,0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,50,Half-Life: Opposing Force,0,0


In [82]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0                     object
 1   user_id           object
 2   items_count       object
 3   steam_id          object
 4   user_url          object
 5   item_id           object
 6   item_name         object
 7   playtime_forever  object
 8   playtime_2weeks   object
dtypes: object(9)
memory usage: 353.8+ MB


In [83]:
# Elimino la primera columna que había quedado "repitiendo" el índice

df_users_items = df_users_items.drop(columns=[''], axis=1)

In [84]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       object
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  object
 7   playtime_2weeks   object
dtypes: object(8)
memory usage: 314.5+ MB


In [87]:
# Eliminar registros donde todos sus valores sean nulos

df_users_items = df_users_items.dropna(how='all')

In [88]:

# Reemplazar los valores vacios por 'NA'

df_users_items.replace('', pd.NA, inplace=True)

In [89]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       object
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  object
 7   playtime_2weeks   object
dtypes: object(8)
memory usage: 314.5+ MB


In [90]:
# Analizo cuantos valores nulos quedaron por columna luego del proceso de transoformación

nulos_users_items = (df_users_items.isnull().mean() * 100).round(2)
nulos_users_items

user_id             98.29
items_count         98.29
steam_id            98.29
user_url            98.29
item_id              0.00
item_name            0.00
playtime_forever     0.00
playtime_2weeks      0.00
dtype: float64

In [91]:
# Debido a su relevancia, se cambian los tipos de datos en las columnas 'item_id', 'playtime_forever' y 'playtime_2weeks'

df_users_items['item_id'] = df_users_items['item_id'].astype(int)

df_users_items['playtime_forever'] = df_users_items['playtime_forever'].astype(int)

df_users_items['playtime_2weeks'] = df_users_items['playtime_2weeks'].astype(int)

In [92]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       object
 2   steam_id          object
 3   user_url          object
 4   item_id           int64 
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 314.5+ MB


In [93]:
# Se exportan los resultados nuevamente a un CSV 

df_users_items.to_csv('/Users/mlucchesi/Henry/PI/data/csv/procesados/users_items_procesado.csv')